# Plot RMS changes 

This notebook will be plotting the differences in RMS due to a synthetic magma body at depth

In [1]:
from pathlib import Path
from mtpy import MTData, MTCollection
from mtpy.modeling.plots.plot_modem_rms import PlotRMS
from mtpy.modeling.modem.data import Data
import numpy as np

In [2]:
h5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests_02.h5")

In [12]:
survey_names = [
    "10_ohmm",
    "1_ohmm",
    "30_ohmm",
    "data",
    "inv_z03_t02_c02_040_11",
    "test_10km",
    "test_11km",
    "test_5km",
    "test_6km",
    "test_7km",
    "test_8km",
    "test_9km",
]

In [9]:
with MTCollection() as mc:
    mc.open_collection(h5_path)
    print(mc.master_dataframe.survey.unique())

['10_ohmm' '1_ohmm' '30_ohmm' 'data' 'inv_z03_t02_c02_040_11' 'test_10km'
 'test_11km' 'test_5km' 'test_6km' 'test_7km' 'test_8km' 'test_9km']
25:09:26T12:19:19 | INFO | line:769 |mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests_02.h5


In [13]:
md_dict = {}
with MTCollection() as mc:
    mc.open_collection(h5_path)
    for survey_id in survey_names:
        print(survey_id)
        mc.working_dataframe = mc.master_dataframe[mc.master_dataframe.survey==survey_id]
        md_dict[survey_id] = mc.to_mt_data().to_dataframe()

10_ohmm
1_ohmm
30_ohmm
data
inv_z03_t02_c02_040_11
test_10km
test_11km
test_5km
test_6km
test_7km
test_8km
test_9km
25:09:26T12:48:12 | INFO | line:769 |mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests_02.h5


In [14]:
def calculate_rms(data_df, model_df):
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        with np.errstate(divide="ignore", invalid="ignore"):
            model_df[f"rms_{col.replace('_', '')}"] = np.abs(
                data_df[col] - model_df[col]
            ) / (np.real(data_df[f"{col}_model_error"]) * np.sqrt(2))
    return model_df

In [15]:
def calculate_rms_change(model_01_df, model_02_df):
    """ 
    Assumes 1 is the original, 2 is the new and returns model 2 as 2 - 1

    In that sense if the value is positive then 2 is larger rms and if
    the value is negative then 2 got smaller.
    """
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        key = f"rms_{col.replace('_', '')}"
        with np.errstate(divide="ignore", invalid="ignore"):
            model_02_df[key] = (model_02_df[key] - model_01_df[key]) / model_02_df[key]

    return model_02_df

In [22]:
data_obj = Data()
data_df = data_obj.read_data_file(Path(
    r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_modem_data_z03_t02_tec_11.dat"
)).dataframe

In [ ]:
data_df

In [26]:
model_data_obj = Data()
model_data_df = data_obj.read_data_file(
    Path(
        r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\clvf_z03_t02_c02_smavg_042.dat"
    )
)

test_data_obj = Data()
test_data_df = data_obj.read_data_file(
    Path(
        r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_outside_test.dat"
    )
)

In [27]:
sorted(md_dict.keys())


['10km_test',
 '5km_test',
 '6km_test',
 '7km_test',
 '9km_test',
 'data',
 'inv_z03_t02_c02_040',
 'melt_test']

In [23]:
md_dict["data"] = data_df

In [24]:
# base_model_01 = calculate_rms(data_df.dataframe, model_data_df.dataframe)
base_model_01 = calculate_rms(md_dict["data"], md_dict["inv_z03_t02_c02_040_11"])

In [25]:
md_dict["data"]

,survey,station,latitude,longitude,elevation,datum_epsg,east,north,utm_epsg,model_east,...,pt_ellipticity_model_error,pt_det,pt_det_error,pt_det_model_error,rms_zxx,rms_zxy,rms_zyx,rms_zyy,rms_tzx,rms_tzy
0,,201,38.818,-122.792,895.999,,0.0,0.0,,-1250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,,201,38.818,-122.792,895.999,,0.0,0.0,,-1250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,,201,38.818,-122.792,895.999,,0.0,0.0,,-1250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,,201,38.818,-122.792,895.999,,0.0,0.0,,-1250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,,201,38.818,-122.792,895.999,,0.0,0.0,,-1250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,,gz366,38.808,-122.757,1140.999,,0.0,0.0,,1750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3122,,gz366,38.808,-122.757,1140.999,,0.0,0.0,,1750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3123,,gz366,38.808,-122.757,1140.999,,0.0,0.0,,1750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3124,,gz366,38.808,-122.757,1140.999,,0.0,0.0,,1750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
import matplotlib.ticker as ticker


# Create a segmented colormap with white band in the middle
def create_segmented_bwr_cmap():
    # Define colors for segments (blue to white to red)
    blues = plt.cm.Blues(np.linspace(0.3, 1, 5))  # 5 shades of blue
    whites = np.array(
        [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]
    )  # 2 white segments for -0.1 to 0.1
    reds = plt.cm.Reds(np.linspace(0, 0.7, 5))  # 5 shades of red

    # Combine the colors
    colors_array = np.vstack([blues[::-1], whites, reds])

    # Create nodes for the segments (ensure we have white at -0.1 to 0.1)
    nodes = np.concatenate(
        [
            np.linspace(-1, -0.1, 6),  # 5 blue segments
            np.linspace(-0.1, 0.1, 2),  # white segment
            np.linspace(0.1, 1, 6),  # 5 red segments
        ]
    )

    # Create the colormap
    # cmap = colors.LinearSegmentedColormap.from_list('SegmentedBWR', list(zip(nodes, colors_array)))
    cmap = colors.LinearSegmentedColormap.from_list("SegmentedBWR", list(colors_array))
    return cmap

In [19]:
rms_cmap = create_segmented_bwr_cmap()

In [ ]:
# test_df = calculate_rms(data_df.dataframe, test_data_df.dataframe)
test_df = calculate_rms(md_dict["data"], md_dict["cl_z03_t02_c02_040_11"])
delta_model_df = calculate_rms_change(base_model_01, test_df)

In [36]:
dp = PlotRMS(delta_model_df)
dp.rms_min = -.1
dp.rms_max = .08
dp.rms_step = .02
dp.fig_num = 4
dp.fig_size = [20, 10]
#dp.rms_cmap = "mt_rd2wh2bl"
dp.rms_cmap = rms_cmap
dp.plot_station = False
dp.box_size = 20
dp.dx = .0175

dp.plot()
dp.save_plot(h5_path.parent.joinpath(f"{'data'}_vs_{'ouside_purged'}_rms.png"), fig_dpi=300)

25:09:24T16:06:52 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))
25:09:24T16:06:55 | INFO | line:125 |mtpy.imaging.mtplot_tools.base | save_plot | Saved figure to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\data_vs_ouside_purged_rms.png


In [26]:
for key in md_dict.keys():
    if key in ["data", "inv_z03_t02_c02_040_11"]:
        continue
    try:
        model_df = calculate_rms(md_dict["data"], md_dict[key])
    except ValueError:
        print(f"Skipping {key}")
        continue
    for base_df, base_id in zip([base_model_01], [40]):
        delta_model_df = calculate_rms_change(base_df, model_df)

        dp = PlotRMS(delta_model_df)
        dp.rms_min = -0.1
        dp.rms_max = 0.08
        dp.rms_step = 0.02
        dp.fig_num = 4
        dp.fig_size = [20, 10]
        # dp.rms_cmap = "mt_rd2wh2bl"
        dp.rms_cmap = rms_cmap
        dp.plot_station = False
        dp.box_size = 20
        dp.dx = 0.0175

        dp.plot()
        dp.save_plot(
            h5_path.parent.joinpath(f"{key}_vs_{base_id}_delta_rms.png"), fig_dpi=300
        )

25:09:26T13:00:15 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))
25:09:26T13:00:18 | INFO | line:125 |mtpy.imaging.mtplot_tools.base | save_plot | Saved figure to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\10_ohmm_vs_40_delta_rms.png
25:09:26T13:00:21 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] cert

In [27]:
for key in md_dict.keys():
    if key in ["data", "inv_z03_t02_c02_040_11"]:
        continue
    try:
        model_df = calculate_rms(md_dict["data"], md_dict[key])
    except ValueError:
        print(f"Skipping {key}")
        continue
    for base_df, base_id in zip([base_model_01], [40]):
        delta_model_df = calculate_rms_change(base_df, model_df)
        print(f"Mean RMS by component: {key}")
        print(
            f"{delta_model_df[['rms_zxx', 'rms_zyy', 'rms_zxy', 'rms_zyx', 'rms_tzx', 'rms_tzy']].mean()}"
        )
        print("\n")
        print(
            f"Mean RMS: {delta_model_df[['rms_zxx', 'rms_zyy', 'rms_zxy', 'rms_zyx', 'rms_tzx', 'rms_tzy']].mean().mean()}"
        )
        print("=" * 20)

Mean RMS by component: 10_ohmm
rms_zxx   -0.003162
rms_zyy    0.001270
rms_zxy    0.001711
rms_zyx   -0.005767
rms_tzx   -0.005482
rms_tzy   -0.000844
dtype: float64


Mean RMS: -0.002045721709061197
Mean RMS by component: 1_ohmm
rms_zxx   -0.003329
rms_zyy    0.001806
rms_zxy    0.002946
rms_zyx   -0.009416
rms_tzx   -0.006500
rms_tzy    0.000233
dtype: float64


Mean RMS: -0.002376557295942562
Mean RMS by component: 30_ohmm
rms_zxx   -0.001989
rms_zyy    0.001181
rms_zxy    0.000572
rms_zyx   -0.002789
rms_tzx   -0.001926
rms_tzy    0.000089
dtype: float64


Mean RMS: -0.0008102736621036347
Mean RMS by component: test_10km
rms_zxx   -0.030017
rms_zyy   -0.005855
rms_zxy   -0.005298
rms_zyx   -0.125062
rms_tzx   -0.009250
rms_tzy    0.001537
dtype: float64


Mean RMS: -0.028990911496660554
Mean RMS by component: test_11km
rms_zxx   -0.007847
rms_zyy   -0.001699
rms_zxy    0.003781
rms_zyx   -0.016369
rms_tzx   -0.002254
rms_tzy   -0.000347
dtype: float64


Mean RMS: -0.004122485502544

In [44]:
key

'lp_melt'

In [47]:
from mtpy.modeling import StructuredGrid3D

In [53]:
model_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo")
mfn_list = (
    list(model_path.glob("cl_gpc*.rho"))
    + list(model_path.glob("*cl_li_17km*.rho"))
    + [model_path.joinpath("cl_test_lp_melt.rho")]
)

In [54]:
mfn_list

[WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_10_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_20_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_30_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_3_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_10_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_30_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_3_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_test_lp_melt.rho')]

In [55]:
s1 = StructuredGrid3D()
s1.from_modem(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_z03_t02_c02_040.rho")

In [56]:
for mfn in mfn_list:
    s2 = StructuredGrid3D()
    s2.from_modem(mfn)
    s1.res_model *= s2.res_model

s1.res_model = s1.res_model **(1/(len(mfn_list) + 1))
s1.to_modem(model_fn=model_path.joinpath("cl_sm_test_avg.rho"))

25:05:15T16:11:23 | INFO | line:873 |mtpy.modeling.structured_mesh_3d | to_modem | Wrote file to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_sm_test_avg.rho
